### To do - August 20th
* Look at the 4 merged df and do additionnal features selection
* Drop or fill NaN values
* Save final df as csv files in output/pre-processing

### Data Frame Merging Roadmap
#### 1. PD Feat
* DIAGFEAT
* EPWORTH
* LNSPD
* REMSLEEP
* SCOPAAUT
* COGATG
* HVLT
* MOCA
* GDSSHORT
* SDM
* STAI
* BENTONOD
* QUIPCS
#### 2. Gen Medical info
* GENPHYEX
* PENEURO
* PENEURO2
* VITAL
#### 3. PD Progression measurements (motors)
* NUPDRS1
* NUPDRS2P
* NUPDR3
* NUPDRS4
* (PASELT)
* MODSEADL
#### 4. Medications data
* PDMEDUSE
* (CURRCOND)
* SURGPD

In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

In [3]:
def number_of_patients(df):
    '''return the number of patients in each csv files'''
    if 'PATNO' in df.columns:
        patients=len(list(set(df['PATNO'])))
    else: 
        patients=0
    return patients

In [4]:
def number_of_events(df): 
    '''return the number of events covered by each csv file'''
    if 'EVENT_ID' in df.columns:
        events=len(list(set(df['EVENT_ID'])))
    else: 
        events=0
    return events

In [5]:
def features_selection(list_df_SEL):
    '''do features selection for each df,sel in list_df_sel (dictionnary)'''
    return 

In [6]:
def patients_selection(df, threshold=6):
    '''return a dataframe containing only the patients with number of visits > threshold'''
    if 'PATNO' in df.columns: 
        visits_number_by_pat=df.groupby('PATNO').size().sort_values(ascending=False)
        mask_sel=visits_number_by_pat>=threshold
        patients_sel=list(mask_sel[mask_sel.values==True].index)
        df=df.loc[df['PATNO'].isin(patients_sel),:]
        df=df.sort_values('PATNO')
        return (patients_sel, df)
    else:
        return 0

In [7]:
def PD_patients_selection(df):
    folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
    files_names=reading_csv_files(folder_path)[1]
    dict_files=reading_csv_files(folder_path)[2]

    ### Patient_Status - actually useful for selecting only the PD patients in each table
    PRODROMA=dict_files[files_names[0]]
    PD_cat=['PD','REGPD','GENPD']
    PD_patients=list(PRODROMA.loc[PRODROMA['RECRUITMENT_CAT'].isin(PD_cat),'PATNO'])
    sub_df=df.loc[df['PATNO'].isin(PD_patients),:]
    return (PD_patients,sub_df)

In [8]:
def table_analysis(list_df,df_names,threshold=6): 
    '''return a dataframe with: 
    -as rows: list of df
    -as columns: 
        - total number of observations=rows
        - total number of features=columns after features_selection - features_selection function 
        - number of events
        - number of PD patients by using PRODROMA info - number_of_patients function
        - number of patients with # of visits > threshold - number_of_patients function
        - shape of final df after selections of rows and columns
        -number of NaN values in the final df
        NB: takes as input the df with features selection. 
    '''   
    # initialisation df
    col=['observations_tot','features','events_num','patients_num','patients_sel','len_df_sel','NaN_values']
    df_df=pd.DataFrame(columns=col,index=df_names)

    # remplissage of dataframe for each row
    for i,df in enumerate(list_df): 
        df_df.iloc[i,:]['observations_tot']=len(df)
        df_df.iloc[i,:]['features']=len(df.columns)
        df_df.iloc[i,:]['events_num']=number_of_events(df)
        df_df.iloc[i,:]['patients_num']=number_of_patients(df)
        
        if not patients_selection(df)==0:
            df=patients_selection(df,threshold=threshold)[1]
            df=PD_patients_selection(df)[1]          
            df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
        else: 
            if 'PATNO' in df.columns:
                df=PD_patients_selection(df)[1]
                df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
            else:
                df_df.iloc[i,:]['patients_sel']=0            
        df_df.iloc[i,:]['len_df_sel']=len(df)
        df_df.iloc[i,:]['NaN_values']=(df.isnull().sum().sum())/(df_df.iloc[i,:]['len_df_sel']*df_df.iloc[i,:]['features'])
        
    return df_df

In [31]:
def INFODT_date(df):
    if 'INFODT' in df.columns: 
        df['INFODT_date']=df['INFODT'].apply(lambda x: datetime.strptime(x,'%m/%Y'))
    return df

In [44]:
def merge(df_left,df_right,on=['PATNO','EVENT_ID','INFODT'],how='inner'): 
    df=pd.merge(df_left,df_right,on=on,how=how)
    return df

In [45]:
def merge_multiple_df(list_df): 
    ## Merging the PD features
    merged_df=merge(list_df[0],list_df[1])
    print((len(merged_df)),len(set(merged_df['PATNO'])))
    for df in list_df[2:]: 
        merged_df=merge(merged_df,df)
        print((len(merged_df)),len(set(merged_df['PATNO'])))
    return merged_df

## 5-Medical History
### PDFEAT is important - keep it.

In [36]:
## 5-Medical History
### PDFEAT is important - keep it.

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Diagnostic_Features.csv': useful: Feat-PD - 1670 patients.
DIAGFEAT=dict_files[files_names[1]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFCRSCM', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFTREMCM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFRIGCM', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFABRCM', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPGCM', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHYPCM', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
DIAGFEAT_sel=DIAGFEAT[SEL]

### 'General_Medical_History.csv': useful: Feat-Gen - 2067 patients
#### NOT TS DATA...
MHXGEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MHCAT', 'MHROW', 'MHHX', 'MHACTRES', 'MHDIAGYR', 
     'PT_CODE', 'PT_NAME', 'HLT_NAME', 'SOCABBR1']
# I need probably to select less features here
MHXGEN_sel=MHXGEN[SEL]

### General_Physical_Exam: useful: Feat-Gen - 2047 patients
GENPHYEX=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PESEQ', 'PECAT', 'ABNORM']
GENPHYEX_sel=GENPHYEX[SEL]

### 'General_Neurological_Exam.csv': useful: Feat-Gen - 2048 patients
PENEURO=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MSRARSP', 'MSRACM', 'MSLARSP', 'MSLACM', 'MSRLRSP', 'MSRLCM',
       'MSLLRSP', 'MSLLCM', 'COFNRRSP', 'COFNRCM', 'COFNLRSP', 'COFNLCM',
       'COHSRRSP', 'COHSRCM', 'COHSLRSP', 'COHSLCM', 'SENRARSP', 'SENRACM',
       'SENLARSP', 'SENLACM', 'SENRLRSP', 'SENRLCM', 'SENLLRSP', 'SENLLCM',
       'RFLRARSP', 'RFLRACM', 'RFLLARSP', 'RFLLACM', 'RFLRLRSP', 'RFLRLCM',
       'RFLLLRSP', 'RFLLLCM', 'PLRRRSP', 'PLRRCM', 'PLRLRSP', 'PLRLCM']
PENEURO_sel=PENEURO[SEL]

### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD - 2049 patients
PENEURO2=dict_files[files_names[13]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
PENEURO2_sel=PENEURO2[SEL]

### 'PD_Features.csv': useful: Feat-PD - caution: these are actually not timeseries but examination only done at screening (SC), BL & V04
### 1017 patients
PDFEAT=dict_files[files_names[14]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'SXMO',
       'SXYEAR', 'PDDXDT', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX','DOMSIDE']
PDFEAT_sel=PDFEAT[SEL]
#list(set(PDFEAT_sel['DXOTHCM'])) - see if we remove the comment feature or not.

### 'Vital_Signs.csv': useful: Feat-Gen
### 2047 patients
VITAL=dict_files[files_names[15]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'WGTKG', 'HTCM', 'TEMPC', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
VITAL_sel=VITAL[SEL]

## 6-Motor-Assessments - all of them can be useful. 
#* UPDRS1 assesses non-motor experiences of daily-living 
#* UPDRS2P assesses motor experiences of daily living
#* UPDRS3 assesses motor signs of PD. Includes Hoehn & Yahr Stage. 
#* UPDRS4 assesses motor complications, dyskinesias, & moto fluctuations using historical & objective information.

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv' - 2011 patients
NUPDR1SP=dict_files[files_names[0]]

### 'MDS_UPDRS_Part_I.csv' - 2010 patients - 
NUPDRS1=dict_files[files_names[2]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT',
       'NP1DDS']
NUPDRS1_sel=NUPDRS1[SEL]

### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv' - 2011 patients
NUPDRS2P=dict_files[files_names[11]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES',
       'NP2HYGN', 'NP2HWRT', 'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE',
       'NP2WALK', 'NP2FREZ']
NUPDRS2P_sel=NUPDRS2P[SEL]

### 'MDS_UPDRS_Part_III.csv' - 2010 patients
NUPDRS3=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CMEDTM', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU',
       'NP3RIGLU', 'PN3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR',
       'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR',
       'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR',
       'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU',
       'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES',
       'DYSKIRAT', 'NHY', 'ANNUAL_TIME_BTW_DOSE_NUPDRS', 'ON_OFF_DOSE',
       'PD_MED_USE']
NUPDRS3_sel=NUPDRS3[SEL]

### 'MDS_UPDRS_Part_IV.csv' - 860 patients
NUPDRS4=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP4WDYSK', 'NP4DYSKI', 'NP4OFF', 'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN']
NUPDRS4_sel=NUPDRS4[SEL]

### 'PASE_-_Household_Activity.csv' - 1248 patients
PASE=dict_files[files_names[3]]

### 'PASE_-_Leisure_Time_Activity.csv' - 1248 patients
PASELT=dict_files[files_names[10]]

### 'Modified_Schwab_+_England_ADL.csv' - 1837 patients
MODSEADL=dict_files[files_names[6]]

## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv' - 1799 patients - OLFACTION
UPSIT=dict_files[files_names[0]]
UPSIT.head()

### 'Epworth_Sleepiness_Scale.csv - 1343 patients - SLEEP. 
EPWORTH=dict_files[files_names[2]]
EPWORTH.head()

### 'Letter_-_Number_Sequencing__PD_.csv' - 1341 patients. - MEMORY. 
LNSPD=dict_files[files_names[3]]
LNSPD.head()

### 'Olfactory_UPSIT.csv' - 4468 patients - OLFACTION. 
#### NB: only take the 2 last columns as features (TOTAL_CORRECT, UPSIIT_PRCNTGE)
OLFACT=dict_files[files_names[4]]
col_old=list(OLFACT.columns)
col_new=['PAT_NO', 'PROTOCOL_ID', 'INFODT', 'SCENT_10_RESPONSE',
       'SCENT_09_RESPONSE', 'SCENT_08_RESPONSE', 'SCENT_07_RESPONSE',
       'SCENT_06_RESPONSE', 'SCENT_05_RESPONSE', 'SCENT_04_RESPONSE',
       'SCENT_03_RESPONSE', 'SCENT_02_RESPONSE', 'SCENT_01_RESPONSE',
       'SCENT_20_RESPONSE', 'SCENT_19_RESPONSE', 'SCENT_18_RESPONSE',
       'SCENT_17_RESPONSE', 'SCENT_16_RESPONSE', 'SCENT_15_RESPONSE',
       'SCENT_14_RESPONSE', 'SCENT_13_RESPONSE', 'SCENT_12_RESPONSE',
       'SCENT_11_RESPONSE', 'SCENT_30_RESPONSE', 'SCENT_29_RESPONSE',
       'SCENT_28_RESPONSE', 'SCENT_27_RESPONSE', 'SCENT_26_RESPONSE',
       'SCENT_25_RESPONSE', 'SCENT_24_RESPONSE', 'SCENT_23_RESPONSE',
       'SCENT_22_RESPONSE', 'SCENT_21_RESPONSE', 'SCENT_40_RESPONSE',
       'SCENT_39_RESPONSE', 'SCENT_38_RESPONSE', 'SCENT_37_RESPONSE',
       'SCENT_36_RESPONSE', 'SCENT_35_RESPONSE', 'SCENT_34_RESPONSE',
       'SCENT_33_RESPONSE', 'SCENT_32_RESPONSE', 'SCENT_31_RESPONSE',
       'SCENT_10_CORRECT', 'SCENT_09_CORRECT', 'SCENT_08_CORRECT',
       'SCENT_07_CORRECT', 'SCENT_06_CORRECT', 'SCENT_05_CORRECT',
       'SCENT_04_CORRECT', 'SCENT_03_CORRECT', 'SCENT_02_CORRECT',
       'SCENT_01_CORRECT', 'SCENT_20_CORRECT', 'SCENT_19_CORRECT',
       'SCENT_18_CORRECT', 'SCENT_17_CORRECT', 'SCENT_16_CORRECT',
       'SCENT_15_CORRECT', 'SCENT_14_CORRECT', 'SCENT_13_CORRECT',
       'SCENT_12_CORRECT', 'SCENT_11_CORRECT', 'SCENT_30_CORRECT',
       'SCENT_29_CORRECT', 'SCENT_28_CORRECT', 'SCENT_27_CORRECT',
       'SCENT_26_CORRECT', 'SCENT_25_CORRECT', 'SCENT_24_CORRECT',
       'SCENT_23_CORRECT', 'SCENT_22_CORRECT', 'SCENT_21_CORRECT',
       'SCENT_40_CORRECT', 'SCENT_39_CORRECT', 'SCENT_38_CORRECT',
       'SCENT_37_CORRECT', 'SCENT_36_CORRECT', 'SCENT_35_CORRECT',
       'SCENT_34_CORRECT', 'SCENT_33_CORRECT', 'SCENT_32_CORRECT',
       'SCENT_31_CORRECT', 'TOTAL_CORRECT', 'UPSIIT_PRCNTGE']
col=dict(zip(col_old,col_new))
OLFACT.rename(columns=col,inplace=True)
SEL=['PAT_NO', 'PROTOCOL_ID', 'INFODT','TOTAL_CORRECT', 'UPSIIT_PRCNTGE']
OLFACT_sel=OLFACT[SEL]

### 'REM_Sleep_Disorder_Questionnaire.csv' - 1810 patients - SLEEP. 
REMSLEEP=dict_files[files_names[5]]
REMSLEEP.head()

### 'Cognitive_Assessments.csv' - 1172 patients
COGTIME=dict_files[files_names[6]]
COGTIME.head()

### 'SCOPA-AUT.csv' - 1345 patients - AUTOMATIC DYSFUNCTIONS
SCOPAAUT=dict_files[files_names[7]]
SCOPAAUT.head()

### Cognitive_Categorization.csv' - 1302 patients - DEMENTIA/MILD COGNITIVE IMPAIRMENT
COGCATG=dict_files[files_names[8]]
COGCATG.head()

### 'Hopkins_Verbal_Learning_Test.csv' - 1341 patients - MEMORY
HVLT=dict_files[files_names[9]]
HVLT.head()

### 'Montreal_Cognitive_Assessment__MoCA_.csv' - 2054 patients - MILD COGNITIVE IMPAIRMENT - PD 
MOCA=dict_files[files_names[10]]
MOCA.head()

### 'Semantic_Fluency.csv' - 1342 patients - Lexical/Semantic Fluency
SFT=dict_files[files_names[11]]
SFT.head()

### 'Geriatric_Depression_Scale__Short_.csv' - 1478 patients - DEPRESSION
GDSSHORT=dict_files[files_names[12]]
GDSSHORT.head()

### 'Symbol_Digit_Modalities.csv' - 1340 patients - COGNITIVE IMPAIRMENT. 
SDM=dict_files[files_names[13]]
SDM.head()

### 'State-Trait_Anxiety_Inventory.csv' - 1474 patients - DEPRESSION
STAI=dict_files[files_names[14]]
STAI.head()

### 'Benton_Judgment_of_Line_Orientation.csv' - 1341 patients - SPATIAL PERCEPTION/ORIENTATION 
BENTONOD=dict_files[files_names[15]]
BENTONOD.head()

### 'QUIP_Current_Short.csv' - 1346 patients - COMPULSIVE DISORDERS
QUIPCS=dict_files[files_names[16]]

In [47]:
## Merging the PD features
PD_feat_df=[DIAGFEAT_sel,EPWORTH,LNSPD,REMSLEEP,SCOPAAUT,COGCATG,HVLT,MOCA,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]
PD_features=merge(PD_feat_df[0],PD_feat_df[1])
print((len(PD_features)),len(set(PD_features['PATNO'])))
for df in PD_feat_df[2:]: 
    PD_features=merge(PD_features,df)
    print((len(PD_features)),len(set(PD_features['PATNO'])))
#PD_features.head()

3644 1105
3023 985
3022 985
3019 985
3006 985
3002 985
2926 980
2924 980
2917 978
2910 978
2904 977
2899 976


## PD Features df

In [50]:
PD_feat_df=[DIAGFEAT_sel,EPWORTH,LNSPD,REMSLEEP,SCOPAAUT,COGCATG,HVLT,MOCA,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]
PD_features=merge_multiple_df(PD_feat_df)
#PD_features.head()

3644 1105
3023 985
3022 985
3019 985
3006 985
3002 985
2926 980
2924 980
2917 978
2910 978
2904 977
2899 976


## General Medical Records df

In [61]:
Medical_Gen_df=[GENPHYEX_sel,PENEURO_sel,PENEURO2_sel,VITAL_sel]
Medical_General=merge_multiple_df(Medical_Gen_df)

24349 2044
24349 2044
21632 1798


## PD Progression Measurements df

In [54]:
PD_Progression_df=[NUPDRS1_sel,NUPDRS2P_sel,NUPDRS3_sel,NUPDRS4_sel,MODSEADL]
# Add PASELT? : if so you need to add a 'INFODT'field...
PD_Progression_motor=merge_multiple_df(PD_Progression_df)

11206 2009
12639 2007
5941 858
5900 841


In [35]:
list_df=[DIAGFEAT_sel,MHXGEN_sel,GENPHYEX_sel,PENEURO_sel,PENEURO2_sel,VITAL_sel,
NUPDRS1,NUPDRS2P,NUPDRS3,NUPDS4,PASE,PASELT,MODSEADL,
UPSIT,EPWORTH,LNSPD,OLFACT_sel,REMSLEEP,COGTIME,SCOPAAUT,COGCATG,HVLT,
MOCA,SFT,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]
df_names=['DIAGFEAT','MHXGEN','GENPHYEX','PENEURO','PENEURO2','VITAL',
'NUPDRS1','NUPDRS2P','NUPDRS3','NUPDRS4','PASE','PASELT','MODSEADL',
'UPSIT','EPWORTH','LNSPD','OLFACT','REMSLEEP','COGTIME','SCOPAAUT','COGCATG','HVLT',
'MOCA','SFT','GDSSHORT','SDM','STAI','BENTONOD','QUIPCS']
dataset_analysis_maintables=table_analysis(list_df,df_names)
#dataset_analysis_maintables

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


## Medications data - to be used later on.

In [62]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Use of PD Medication: useful: Meds - but not to be used for now.
PDMEDUSE=dict_files[files_names[4]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG', 'ONOTHER', 'FULNUPDR', 'PDMEDDT',
       'PDMEDTM', 'NUPDRTM']
PDMEDUSE_sel=PDMEDUSE[SEL]

### 'Concomitant_Medications.csv': useful: Meds - no time to exploit this stuff. 
CMED=dict_files[files_names[6]]

### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
CURRCOND.columns
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'SEQNO',
       'CONDCAT', 'DIAGYR','CONDTERM', 'RESOLVD', 'RESYR',
       'HLGTNAME', 'HLT_NAME','PT_CODE','SOCABBR1', 'VMEDDRA']
CURRCOND_sel=CURRCOND[SEL]

### 'Adverse_Event_Log.csv': useful: Meds - but not to be used for now.
#### Measures the adverse events occurring after LUMBAR PUNCTURE, SPECT, AV-133 PET Imaging, 
#florbetaben imaging, skin biopsy.  -USELESS

AE=dict_files[files_names[9]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'AESEQ',
       'AETERM', 'STARTDT','STOPDT', 'AESEVER', 'SAE',
       'AERELAT', 'RELDSCAN', 'RELLP', 'RELPRCDR', 'RELAV133', 'RELFB',
       'RELSKBIO', 'AEOUTCOM', 'AEFLUP',
       'PT_CODE', 'PT_NAME', 'HLT_NAME', 'HLGTNAME', 'SOCABBR1', 'VMEDDRA']
AE_sel=AE[SEL]

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.
SURGPD=dict_files[files_names[12]]
SURGPD.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDSURG', 'PDSURGDT', 'PDSURGTP']
SURGPD_sel=SURGPD[SEL]

## Medications data df - 0.43 % of NaN values!! > use fillna values

In [65]:
medications_df=[PDMEDUSE,SURGPD]
medications_data=merge_multiple_df(medications_df)
medications_data.isnull().sum().sum()/(len(medications_data)*len(medications_data.columns))

3109 800


0.43799834097949925

In [55]:
list_df=[PDMEDUSE_sel,CURRCOND_sel,AE_sel,SURGPD_sel]
df_names=['PDMEDUSE','CURRCOND','AE','SURGPD']
dataset_analysis_meds=table_analysis(list_df,df_names)
#dataset_analysis_meds

# Additionnal data that will be used later on (imaging data and medication data)

## 4-Imaging - dataset not to be used for the time being.
### Info about Imaging Data pages 98 to 101 in the study protocol. 

In [17]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### FBB_Analysis_Data.csv
FBBANAL=dict_files[files_names[0]]
# no selection to do. 

### AV-133_Imaging.csv
AVIMAG=dict_files[files_names[2]]
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'PETCMPLT','VMATRSLT']
AVIMAG_sel=AVIMAG[SEL]

### DaTscan_Imaging.csv
DATSCAN=dict_files[files_names[3]]
DATSCAN.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DATSCAN','VSINTRPT','VSRPTELG']
DATSCAN_sel=DATSCAN[SEL]

### AV-133_SBR_Results.csv
ind_av133_sbr=dict_files[files_names[5]]
col_old=list(ind_av133_sbr.columns)
col_new=['PATNO',
 'EVENT_ID',
 'INFODT',
 'TIMEPOINT',
 'RCAUD-S',
 'RPUTANT-S',
 'RPUTPOST-S',
 'LCAUD-S',
 'LPUTANT-S',
 'LPUTPOST-S',
 'update_stamp']
col=dict(zip(col_old,col_new))
ind_av133_sbr.rename(columns=col,inplace=True)
SEL=['PATNO', 'EVENT_ID', 'INFODT', 'RCAUD-S', 'RPUTANT-S',
       'RPUTPOST-S', 'LCAUD-S', 'LPUTANT-S', 'LPUTPOST-S']
ind_av133_sbr_sel=ind_av133_sbr[SEL]

### Magnetic_Resonance_Imaging.csv
MRI=dict_files[files_names[6]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MRICMPLT', 'MRIWDTI', 'MRIWRSS', 'MRIRSLT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG']
MRI_sel=MRI[SEL]

### DTI_Regions_of_Interest
DTIROI=dict_files[files_names[7]]
SEL=['PATNO', 'PAG_NAME', 'INFODT', 'Measure','ROI1', 'ROI2',
       'ROI3', 'ROI4', 'ROI5', 'ROI6', 'REF1', 'REF2']
DTIROI_sel=DTIROI[SEL]

### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv - not found in data dictionary...
SPECT_vis_assess=dict_files[files_names[8]]
#SPECT_vis_assess.head()

In [18]:
list_df=[FBBANAL,AVIMAG_sel,DATSCAN_sel,ind_av133_sbr_sel,MRI_sel,DTIROI_sel]
df_names=['FBBANAL','AVIMAG','DATSCAN','ind_av133_sbr','MRI','DTIROI']
dataset_analysis_imaging=table_analysis(list_df,df_names,threshold=3) #threshold=3 to have more 
dataset_analysis_imaging

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
FBBANAL,88,37,6,88,0,0,NaN
AVIMAG,87,7,9,36,16,57,0.105263
DATSCAN,2483,8,16,1045,414,1637,0.213653
ind_av133_sbr,80,9,6,35,14,50,0
MRI,2125,12,16,1326,188,713,0.3241
DTIROI,1200,12,0,263,195,872,0
